In [1]:
"""
C:\Spiele\Dark Projekt\MODS\ep\obj\lcbed.bin
"""

'\nC:\\Spiele\\Dark Projekt\\MODS\\ep\\obj\\lcbed.bin\n'

In [1]:
import math
import re
import struct
import os
from struct import pack, unpack


###
### Helper classes
###

from importlib import reload


import BinNodeExtractor
import NodesImport
reload(NodesImport)
reload(BinNodeExtractor)

from NodesImport import *
from BinNodeExtractor import parse_bin, Norms, FaceImported, AIMesh, StaticModel

if __name__ == "__main__":
    # Default test files for empty import.
    file_path=r"C:\Spiele\Dark Projekt\MYMODS\UPPERMODS\obj\RenderTest2.bin"

    file_path=r"C:\Spiele\Dark Projekt\MYMODS\UPPERMODS\obj\RenderTestbsp2.bin"
    #file_path=r"C:\Spiele\Dark Projekt\MYMODS\UPPERMODS\obj\tree3div.bin"

    z = input("Choose file_path:")
    if z != "":
        if len(z) < 20: # Assume it's just a file name, look in main folders
            file_path = r"C:\Spiele\Dark Projekt\MYMODS\UPPERMODS\obj\\" +z
            file_found = os.path.exists(file_path)
            if not file_found:
                file_path = r"C:\Spiele\Dark Projekt\MODS\ep\obj\\" + z
        else:
            file_path = z # total file path

    binData = open(file_path, 'r+b')
    typeID = binData.read(4)
    binData.seek(0)
    if typeID == b'LGMD':
        object_data = parse_bin(binData)
    elif typeID == b'LGMM':
        object_data = AIMesh.parse_bin(binData)
    else:
        binData.close()
        raise TypeError("Wrong format.")
    binData.close()
    #for i in range((faceOffset-normOffset)//12):
    #    if i not in Norms:
    #        Norms[i] = getNormalByIndex(i)
    
    
    ## PRINT NODES ##
    #C:\Spiele\Dark Projekt\RES\ape02.bin
    #C:\Spiele\Dark Projekt\RES\MecSol01.bin

6
2
4
2
2
2
2
2
2
2
2
16
16
4
4
1
1
1
1
12
16
16
4
4
4
4
4
4
1
1
1
1
12
4
1
1
1
1
12
12
2
2
4
2
1
1
4
4
4
4
4
4
16 <i
64
16 <fff
192
4
4
4
2
16 <H
32
16 <fff
192
16 <f
64
4
4
4
nTorsos bin_struct([('joint', '<i'), ('parent', '<i'), ('num_fixed_points', '<i'), ('joint_id', (16, '<i')), ('pts', (16, '<fff'))])
not def nTorsos <class 'str'>
not defined
not def not defined <class 'str'>
Choose file_path:
mat type 0 handle/argb: 0 uv/ipal (0.0, 0)
mat type 0 handle/argb: 0 uv/ipal (0.0, 0)
mat type 0 handle/argb: 0 uv/ipal (0.0, 0)
mat type 0 handle/argb: 0 uv/ipal (0.0, 0)
mat type 0 handle/argb: 0 uv/ipal (0.0, 0)
mat type 0 handle/argb: 0 uv/ipal (0.0, 0)
File version is 4


ImportError: attempted relative import with no known parent package

In [12]:
"""
Display the Nodes as a Tree. Needs igraph and pyplot
https://igraph.org/
https://plotly.com/python/tree-plots/
"""

try:
    import igraph
    import plotly
except ImportError:
    print("igraph or plotly are not installed", 
          "https://igraph.org/", 
          "https://plotly.com/python/tree-plots/", sep="\n")
else:
    import numpy as np

    Nodes = Node.AllNodes
    Nodes.sort(key=lambda n: n.nd_index)

    from igraph import Graph
    nr_vertices = len(Nodes)
    #G = Graph.Tree(nr_vertices, 2) # 2 stands for children number
    G = Graph(n=nr_vertices, directed=True)

    #v_label = list(map(str, range(nr_vertices)))
    v_label = [("(" + str(n.nd_index) + ") " + n.__class__.__name__) for n in Node.AllNodes]

    def NodeFacesToStr(n):
        s = ""
        if not hasattr(n, "faces"):
            return s
        s += "\n"
        if hasattr(n, "pgons_before"):
            i_max = n.pgons_before
        else:
            i_max = len(n.faces)
        for i in range(i_max):
            s+= str(n.faces[i]) +", "
        if i_max < len(n.faces):
            s+= " || "
            for i in range(i_max, len(n.faces)):
                s+= str(n.faces[i]) +", "
        return s[:-2]


    labels = [NodeFacesToStr(n) for n in Nodes]
    es_labels = []
    nd_types = [""] * nr_vertices
    nd_normals = [""] * nr_vertices
    nd_normal_ids = [""] * nr_vertices
    nd_d = [""] * nr_vertices

    for node in Nodes:
        for k,v in node.__dict__.items():
            if isinstance(v, Node) and k != "parent":
                G.add_edges([(node.nd_index, v.nd_index)])
                nd_types[v.nd_index] = k
        if type(node) == Node_split:
            nd_normals[node.nd_index] = f"{node.normal[0]:.3f}, {node.normal[1]:.3f}, {node.normal[2]:.3f}"
            nd_d[node.nd_index] = f"{node.d:.3f}"
            nd_normal_ids[node.nd_index] =node.norm_id

    data = (np.vstack(v_label), 
            np.vstack(nd_types), 
            np.vstack(labels),
            np.vstack(nd_normals),
            np.vstack(nd_d),
            np.vstack(nd_normal_ids)
           )        

    lay = G.layout('rt')

    position = {k: lay[k] for k in range(nr_vertices)}
    Y = [lay[k][1] for k in range(nr_vertices)]
    M = max(Y)

    #es = EdgeSeq(G) # sequence of edges
    E = [e.tuple for e in G.es] # list of edges
    L = len(position)
    Xn = [position[k][0] for k in range(L)]
    Yn = [2*M-position[k][1] for k in range(L)]
    Xe = []
    Ye = []
    for edge in E:
        Xe+=[position[edge[0]][0],position[edge[1]][0], None]
        Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]


    import plotly.graph_objects as go
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=Xe,
                       y=Ye,
                       mode='lines',
                        name='edge',
                       line=dict(color='rgb(210,210,210)', width=1),
                       text=es_labels,
                       hoverinfo='text',
                       opacity = 1
                       ))
    fig.add_trace(go.Scatter(x=Xn,
                      y=Yn,
                      mode='markers',
                      name ="",
                      marker=dict(symbol='square',
                                    size=40,
                                    color='#6175c1',    #'#DB4551',
                                    line=dict(color='rgb(50,50,50)', width=1)
                                    ),
                      text=labels,
                      #hoverinfo='text',
                      opacity=0.8,
                      customdata = np.dstack(data),
                      hovertemplate = "%{customdata[0][0]} - %{customdata[0][1]}<br>"
                                     +"-----<br>%{customdata[0][2]}"
                                     +"<br>norm(%{customdata[0][5]}):\t%{customdata[0][3]}"
                                     +"<br>\td:\t\t%{customdata[0][4]}"
                      ))

    def make_annotations(pos, text, font_size=10, font_color='rgb(250,250,250)'):
        L=len(pos)
        if len(text)!=L:
            raise ValueError('The lists pos and text must have the same len')
        annotations = []
        for k in range(L):
            annotations.append(
                dict(
                    text=v_label[k], # or replace labels with a different list for the text within the circle
                    x=pos[k][0], y=2*M-position[k][1],
                    xref='x1', yref='y1',
                    font=dict(color=font_color, size=font_size),
                    showarrow=False)
            )
        return annotations

    axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
                zeroline=False,
                showgrid=False,
                showticklabels=False,
                )

    fig.update_layout(title= file_path,
                  annotations=make_annotations(position, v_label),
                  font_size=12,
                  showlegend=False,
                  xaxis=axis,
                  yaxis=axis,
                  margin=dict(l=40, r=40, b=85, t=100),
                  hovermode='closest',
                  plot_bgcolor='rgb(0,0,0)'
                  )
    fig.show()

In [13]:
for id, faces in Norms.items():
    print(id, faces)

0 [0, 11, 12, 18, 19, 20, 21, 'split d=-1.000']
1 [1, 10, 13, 22, 23, 24, 25, 'split d=-1.000']
2 [2, 6, 17, 27, 32, 37, 38]
3 [3, 7, 16, 26, 28, 36, 39, 'split d=-1.000']
4 [4, 8, 15, 29, 33, 35, 40]
5 [5, 9, 14, 30, 31, 34, 41]


In [14]:
# Coplanar faces need heavy rounding

keys = list(FaceImported.coplanar.keys())
keys.sort(key=lambda k: k[3])

for k in keys:
    print(f"{str(k[:3]):<22} d: {str(k[3])}: ", FaceImported.coplanar[k])

(0.0, 0.0, -1.0)       d: -7.0:  [12]
(0.0, 0.0, 1.0)        d: -7.0:  [13]
(-1.0, 0.0, 0.0)       d: -7.0:  [14]
(0.0, -1.0, 0.0)       d: -7.0:  [15]
(1.0, 0.0, 0.0)        d: -7.0:  [16]
(0.0, 1.0, 0.0)        d: -7.0:  [17]
(0.0, 1.0, 0.0)        d: -1.0:  [6, 27, 32, 37, 38]
(1.0, 0.0, 0.0)        d: -1.0:  [7, 26, 28, 36, 39]
(0.0, -1.0, 0.0)       d: -1.0:  [8, 29, 33, 35, 40]
(-1.0, 0.0, 0.0)       d: -1.0:  [9, 30, 31, 34, 41]
(0.0, 0.0, 1.0)        d: -1.0:  [10, 22, 23, 24, 25]
(0.0, 0.0, -1.0)       d: -1.0:  [11, 18, 19, 20, 21]
(0.0, 0.0, -1.0)       d: 5.0:  [0]
(0.0, 0.0, 1.0)        d: 5.0:  [1]
(0.0, 1.0, 0.0)        d: 5.0:  [2]
(1.0, 0.0, 0.0)        d: 5.0:  [3]
(0.0, -1.0, 0.0)       d: 5.0:  [4]
(-1.0, 0.0, 0.0)       d: 5.0:  [5]
